# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.915552236396898),
 (1, 1.915552236396898),
 (2, 1.915552236396898),
 (3, 1.915552236396898),
 (4, 1.915552236396898)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('is', 18), ('it', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.07358143672056194),
   (None, 0.07465376703811777),
   (None, 0.08071369019639307),
   (None, 0.10671284455226326),
   (None, 0.10847301060179959),
   (None, 0.13309064522473935),
   (None, 0.15062114701305285),
   (None, 0.19599548926917032),
   (None, 0.2314673861303892),
   (None, 0.3126482117740087),
   (None, 0.34452124268219897),
   (None, 0.37836011512893786),
   (None, 0.3807599632632003),
   (None, 0.38520021352684186),
   (None, 0.4095249609436177),
   (None, 0.4271315895267198),
   (None, 0.4719499079078927)]),
 (1,
  [(None, 0.5728241089972439),
   (None, 0.65464375517994),
   (None, 0.687923026700727),
   (None, 0.7560262576255459),
   (None, 0.771363324927146),
   (None, 0.8121052633768672),
   (None, 0.8275350498698315),
   (None, 0.830056462570637),
   (None, 0.850241437535169),
   (None, 0.8705549667235577),
   (None, 0.9269288197341592),
   (None, 0.9714502467750076),
   (None, 0.9822061890144934)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
numbers = [5, -25, 0, 38]

def RECORDREADER():
  return [(0, numbers[n]) for n in range (len(numbers))]

def MAP(n, num):
  yield (n, num)

def REDUCE(n, num):
  yield (n, max(num))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 38)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
numbers = [5, -25, 0, 38]

def RECORDREADER():
  return [(0, numbers[n]) for n in range (len(numbers))]

def MAP(n, num):
  yield (n, num)

def REDUCE(n, num):
  yield (n, sum(num) / len(num))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 4.5)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def groupbykey(iterable):
  t = {}
  iterable = sorted(iterable, key=lambda k: k[0])
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

#проверила на примере среднего
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 4.5)]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='female', social_contacts=260),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=33, gender='female', social_contacts=600),
    User(id=3, age=25, gender='female', social_contacts=800),
]

In [38]:
maps = 3
reducers = 3

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

def MAP(_, user: str):
    yield (user.id, user)

# Из всех пользователей с одинаковым id остается только тот, который встретился первым
def REDUCE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=55, social_contacts=20, gender='male'),
   User(id=3, age=33, social_contacts=800, gender='female')]),
 (1, [User(id=1, age=25, social_contacts=240, gender='female')]),
 (2, [User(id=2, age=25, social_contacts=500, gender='female')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [39]:
# Предикат C - проверка параметра gender
def MAP(_, row: NamedTuple):
    if row.gender == "female":
      #ключ и значение д.б. одинаковы
        yield (row, row)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  [User(id=1, age=25, social_contacts=240, gender='female')]),
 (User(id=1, age=25, social_contacts=260, gender='female'),
  [User(id=1, age=25, social_contacts=260, gender='female')]),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  [User(id=2, age=25, social_contacts=500, gender='female')]),
 (User(id=3, age=25, social_contacts=800, gender='female'),
  [User(id=3, age=25, social_contacts=800, gender='female')]),
 (User(id=3, age=33, social_contacts=600, gender='female'),
  [User(id=3, age=33, social_contacts=600, gender='female')]),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  [User(id=3, age=33, social_contacts=800, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [52]:
input_collection = [
    dict(id=0, age=54, gender="male", social_contacts=220),
    dict(id=0, age=25, gender="female", social_contacts=210),
    dict(id=1, age=27, gender="middle", social_contacts=652),
    dict(id=2, age=16, gender="male", social_contacts=12),
    dict(id=2, age=35, gender="middle", social_contacts=240),
    dict(id=3, age=31, gender="male", social_contacts=520),
    dict(id=3, age=32, gender="female", social_contacts=431),
    dict(id=3, age=36, gender="middle", social_contacts=430)
]

# Группировка по id
def MAP(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # если не мужчина и не женщина, удаляем поле гендера
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u["id"], u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 54, 'gender': 'male', 'social_contacts': 220},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 210}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 652}]),
 (2,
  [{'id': 2, 'age': 16, 'gender': 'male', 'social_contacts': 12},
   {'id': 2, 'age': 35, 'social_contacts': 240}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 520},
   {'id': 3, 'age': 32, 'gender': 'female', 'social_contacts': 431},
   {'id': 3, 'age': 36, 'social_contacts': 430}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [54]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=33, gender="male", social_contacts=973),
]

def MAP(_, row: NamedTuple):
    yield (row.id, row)

#возвращаем только 1 повторение записи
def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=4, age=33, social_contacts=973, gender='male'),
  User(id=4, age=33, social_contacts=973, gender='male'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [56]:
def MAP(_, row: NamedTuple):
    yield (row.id, row)

#возвращаем запись, только если было 2 повторения записи
def REDUCE(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')],
 [User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [61]:
def MAP(collection_id, user):
    yield (user, collection_id)

#возвращаем запись, только если они входят в первую выборку, но не во 2-ую
def REDUCE(user, collections):
    if collections == [0]:
        yield (user)

def RECORDREADER():
    return [(0, a) for a in input_collection_1] + [(1, b) for b in input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [75]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    job_id: int

class Job(NamedTuple):
    id: int
    name: str

users_collection = [
    User(id=1, age=25, gender="male", social_contacts=20, job_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, job_id=2),
    User(id=3, age=22, gender="male", social_contacts=642, job_id=2),
    User(id=4, age=54, gender="female", social_contacts=123, job_id=3),
    User(id=5, age=24, gender="female", social_contacts=800, job_id=1),
]

job_collection = [
    Job(id=1, name="Doctor"),
    Job(id=2, name="Nerce"),
    Job(id=3, name="Teacher"),
    Job(id=4, name="Driver"),
]

def MAP(job_id, row):
    yield (job_id, row)

def REDUCE(job_id, rows):
    users = []
    job = None
    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            job = row
    for row in rows:
        if type(row) is User:
          #(a,b,c), где a - юзер, b - id профессии, c - профессия
            yield (row, row.job_id, job)

def RECORDREADER():
    return [(user.job_id, user) for user in users_collection] + [(job.id, job) for job in job_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=1, age=25, social_contacts=20, gender='male', job_id=1),
  1,
  Job(id=1, name='Doctor')),
 (User(id=5, age=24, social_contacts=800, gender='female', job_id=1),
  1,
  Job(id=1, name='Doctor')),
 (User(id=2, age=25, social_contacts=240, gender='female', job_id=2),
  2,
  Job(id=2, name='Nerce')),
 (User(id=3, age=22, social_contacts=642, gender='male', job_id=2),
  2,
  Job(id=2, name='Nerce')),
 (User(id=4, age=54, social_contacts=123, gender='female', job_id=3),
  3,
  Job(id=3, name='Teacher'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [83]:
def MAP(job_id, user, job):
    yield (job_id, user)

def REDUCE(job_id, rows):
    yield f"job_id={job_id} has {len(rows)} user(s)"

def RECORDREADER():
    return [(job_id, user, job) for user, job_id, job in output]

output1 = MapReduce(RECORDREADER, MAP, REDUCE)
output1 = list(output1)
output1

['job_id=1 has 2 user(s)', 'job_id=2 has 2 user(s)', 'job_id=3 has 1 user(s)']

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [86]:
from typing import Iterator
import numpy as np

n = 4
mat = np.ones((5, n))
vec1 = np.random.rand(n // 2)
#разбиение вектора на векторы поменьше, которые помещаются в памяти
vec2 = np.random.rand(n - n // 2)

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  if (j < len(vec1)):
    yield (i, value * vec1[j])
  else:
    yield (i, value * vec2[j - len(vec1)])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      #сохранение [i, j] элемента в формате ((i, j), key)
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.7240405993658776),
 (1, 1.7240405993658776),
 (2, 1.7240405993658776),
 (3, 1.7240405993658776),
 (4, 1.7240405993658776)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [87]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [89]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i][j] * v1)

def REDUCE(key, values):
  yield (key, sum(values))

Проверьте своё решение

In [90]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [91]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [92]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range (small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i, j), small_mat[i,j]), ((j, k), big_mat[j,k]))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [93]:
maps = 2
reducers = 2

def INPUTFORMAT():
    global maps
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]
print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[1.28736157 1.11313575 1.14804265 1.54741968 1.71980557 1.72765435
  1.85300805 1.38069886 1.35987334 1.36002556 1.54206879 1.77826373
  0.7732507  0.66956891 1.29367764 0.97071314 1.10972224 1.15452789
  0.95199126 1.92123088 0.57723922 1.31697334 1.66721027 1.79195661
  0.63187608 2.19627191 1.32003501 1.40495271 1.60564659 1.81832295
  1.09923706 1.51752895 1.33983171 2.10223303 1.18653038 2.11564203
  0.78999279 1.53908051 1.02512173 1.63305882]
 [0.54256845 0.26410067 0.41129655 0.57538328 0.57554324 0.72016389
  0.73225112 0.43643773 0.30994609 0.4349487  0.6644398  0.55581258
  0.40400026 0.20544326 0.45677011 0.31561965 0.4283815  0.28698984
  0.50415242 0.69492434 0.2150264  0.53331834 0.6254633  0.52401975
  0.33614124 0.74290294 0.55199389 0.50893709 0.72052345 0.63750032
  0.32306447 0.51142953 0.38975837 0.70120341 0.3994841  0.81951573
  0.15770323 0.56235826 0.34907808 0.58810393]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение будет работать